In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f(x,y):
    "Objective function"
    return (x-3.14)**2 + (y-2.72)**2 + np.sin(3*x+1.41) + np.sin(4*y-1.73)

def update(X, V, pbest, pbest_obj, gbest, gbest_obj, w, c1, c2):
    """
    Update function
    """   
    # randomly sample r1 and r2
    r1, r2 = np.random.rand(2)
    
    # update the velocity
    V_new = w * V + c1 * r1 * (pbest - X) + c2 * r2 * (gbest.reshape((2,1)) - X)

    # update the position
    X_new = X + V_new
    
    # update gbest and pbest
    obj = f(X_new[0], X_new[1])
    pbest_new = np.where(obj < pbest_obj, X_new, pbest)
    pbest_obj_new = np.minimum(obj, pbest_obj)

    gbest_candidate_obj = pbest_obj_new.min()
    if gbest_candidate_obj < gbest_obj:
        gbest_obj_new = gbest_candidate_obj
        gbest_new = pbest_new[:, pbest_obj_new.argmin()]
    else:
        gbest_obj_new = gbest_obj
        gbest_new = gbest

    return X_new, V_new, pbest_new, pbest_obj_new, gbest_new, gbest_obj_new
 
def contour_plot_function(f, ax=None):
    if not ax:
        _, ax = plt.subplots()
    
    # Contour plot: With the global minimum showed as "X" on the plot
    x, y = np.array(np.meshgrid(np.linspace(0,5,100), np.linspace(0,5,100)))
    z = f(x, y)
    x_min = x.ravel()[z.argmin()]
    y_min = y.ravel()[z.argmin()]

    ax.imshow(z, extent=[0, 5, 0, 5], origin='lower', cmap='viridis', alpha=0.5)
    # ax.set_colorbar()
    ax.plot([x_min], [y_min], marker='x', markersize=5, color="white")
    contours = ax.contour(x, y, z, 10, colors='black', alpha=0.4)
    ax.clabel(contours, inline=True, fontsize=8, fmt="%.0f")

In [ ]:
# initialize particles
n_particles = 40
np.random.seed(2)
X = np.random.rand(2, n_particles) * 5
V = np.random.randn(2, n_particles) * 0.1

# initialize pbest and gbest
pbest = X
pbest_obj = f(pbest[0], pbest[1])
gbest = pbest[:, pbest_obj.argmin()]
gbest_obj = gbest.min()

# plot the particles
_, ax = plt.subplots(figsize=(6,6), dpi=80)
contour_plot_function(f, ax=ax)
x,y = X[0], X[1]
ax.scatter(x, y, color='b')
ax.scatter(pbest[0], pbest[1], color='k', alpha=0.5)
ax.scatter([gbest[0]], [gbest[1]], marker='s', color='y', alpha=1, zorder=10)
ax.quiver(x, y, V[0], V[1], color='b')
plt.axis('off')
plt.show()

In [ ]:
w = 0.75
c1 = 0.1
c2 = 0.15

n_iterations = 70
plot_iterations = [0, 5, 10, 25, 50, 69]
plot_it = 0
gopt = []
lopt = []
d_to_gopt = []

axes = plt.subplots(len(plot_iterations), 1, figsize=(5, len(plot_iterations)*5), dpi=80)
for i in range(n_iterations):
    X, V, pbest, pbest_obj, gbest, gbest_obj = update(X, V, pbest, pbest_obj, gbest, gbest_obj, w, c1, c2)
    gopt.append(gbest_obj)
    lopt.append(pbest_obj)
    d_to_gopt.append(np.linalg.norm(X - gbest.reshape((2, 1)), axis=0))

    # plot the particles
    # _, ax = plt.subplots(figsize=(6,6), dpi=80)
    if i in plot_iterations:
        ax = axes[1][plot_it]
        contour_plot_function(f, ax=ax)
        x,y = X[0], X[1]
        ax.scatter(x, y, color='b')
        ax.scatter(pbest[0], pbest[1], color='k', alpha=0.2)
        ax.scatter([gbest[0]], [gbest[1]], marker='s', color='y', alpha=1, zorder=10)
        ax.quiver(x, y, V[0], V[1], color='b')
        ax.set_title(f"Iteration {i}")
        plot_it += 1        
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(4,1.5), dpi=100)
plt.plot(range(n_iterations), gopt, label="Global")
plt.plot(range(n_iterations), [np.mean(l) for l in lopt], label="Local (avg.)")
plt.grid()
plt.xlabel("Iteration")
plt.ylabel("Minimum cost")
plt.legend()
plt.show()

plt.figure(figsize=(4,1.5), dpi=100)
plt.plot(range(n_iterations), [np.min(l) for l in d_to_gopt], label="min")
plt.plot(range(n_iterations), [np.mean(l) for l in d_to_gopt], label="avg")
plt.plot(range(n_iterations), [np.max(l) for l in d_to_gopt], label="max")
plt.grid()
plt.xlabel("Iteration")
plt.ylabel("Distance to gOPT")
plt.legend()
plt.show()



